# Libraries

In [72]:
import requests

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None) # display all columns in DF
#pd.set_option('display.max_rows', None) # display all columns in DF

import datetime as dt

import matplotlib.pyplot as plt
import seaborn as sns

from ast import literal_eval

# Uploading raw data

In [73]:
df_openFDA_NDC = pd.read_csv('raw_openFDA_NDC_data.csv', sep = '~')
df_medicaid = pd.read_csv('raw_medicaid_data.csv', sep = '~')
df_PPI = pd.read_excel('PPI industry data for Pharmaceutical preparation manufacturing.xlsx', skiprows = 10)
df_atc = pd.read_excel('raw_atc_data.xlsx')

# Data manipulation: Raw data

### OpenFDA NDC

Clearing the labeler name

In [74]:
df_label = df_openFDA_NDC
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Accord Healthcare Inc.','Accord Healthcare, Inc.','Accord Healthcare, Inc.,','Accord Healthcare','Accord BioPharma Inc.','Accord Healthcare, Inc'], 'Accord Healthcare Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Afaxys Pharma, LLC', 'Afaxys Pharma LLC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('ACI Healthcare USA, Inc.', 'ACI Healthcare USA, Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Alembic Pharmaceuticals, Inc.', 'Alembic Pharmaceuticals Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Alcon Laboratories, Inc.', 'Alcon, Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Almatica Pharma LLC', 'Almatica Pharma Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Amgen USA Inc.', 'Amgen Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('AMICI PHARMACEUTICALS LLC', 'Amici Pharmaceuticals LLC.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Amneal Pharmaceuticals LLC', 'Amneal Pharmaceuticals')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Amryt Pharmaceutcials DAC', 'Amryt Pharmaceuticals DAC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['APNAR PHARMA LP','Apnar Pharma LP'], 'Apnar Pharma')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('APOTEX CORP.', 'Apotex Corp')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Alembic Pharmaceuticals, Inc.', 'Alembic Pharmaceuticals Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Alexion Pharmaceuticals, Inc.', 'Alexion Pharmaceuticals Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Alvogen, Inc.','Alvogen'], 'Alvogen Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Akorn','Akorn, Inc.'], 'Akorn Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Amneal Pharmaceuticals NY LLC','Amneal Pharmaceuticals of New York LLC','Amneal Pharmaceuticals of New York, LLC'], 'Amneal Pharmaceuticals LLC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Areva Pharmaceuticals Inc.','Areva Pharmaceuticals, Inc.','Areva Pharmaceuticals,Inc.'], 'Areva Pharmaceuticals')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Amring Pharmaceuticals, Inc.', 'Amring Pharmaceuticals Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Apotex Corp.', 'Apotex Corp')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Aytu Therapeutics, LLC','Aytu Therapeutics')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Arbor Pharmaceuticals, Inc.', 'Arbor Pharmaceuticals')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Aurolife Pharma, LLC', 'Aurolife Pharma LLC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Baxter Healthcare Corporation','Baxter Healthcare CORP'], 'Baxter Healthcare Company')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Bausch & Lomb Incoporated', 'Bausch & Lomb Incorporated')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Bausch Health US, LLC','Bausch Health Us, LLC','Bausch Health Americas Inc.'], 'Bausch Health US LLC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Bayshore Pharmaceuticals, LLC','Bayshore Pharmaceuticals LLC (NJ)'], 'Bayshore Pharmaceuticals LLC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Biocon Pharma Inc,','Biocomp Pharma, Inc.','Biocon Pharma Inc.','Biocon Pharma Inc.,'], 'Biocon Pharma Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Biogen Inc.', 'Biogen')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Bionpharma Inc.,','BionPharma Inc.,'], 'Bionpharma Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['BluePoint Labortories','BluePoint Laboratories Inc.','BluePoint'], 'BluePoint Laboratories')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Burel Pharmaceuticals, LLC','Burel Pharma'], 'Burel Pharmaceuticals, Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Boehringer Ingelheim Pharmaceuticals, Inc.', 'Boehringer Ingelheim Pharmaceuticals Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Breckenridge Pharmaceutical, Inc.', 'Breckenridge Pharmaceutical, Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Bionpharma Inc.,', 'Bionpharma Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Bristol-Myers Squibb/Sanofi Pharmaceuticals Partnership', 'Bristol-Myers Squibb Pharma Company')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Camber Pharmaceuticals, Inc.','Camber Pharmaceuticals','Camber Pharmaceuticals Inc' ,'Camber Pharmaceuticals, Inc'], 'Camber Pharmaceutical Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Chartwell RX, LLC.', 'Chartwell RX LLC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Cameron Pharmaceuticals, LLC', 'Cameron Pharmaceuticals')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Carlsbad Technology, Inc.', 'Carlsbad Technology, Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Cipla USA Inc.,','CIPLA USA, Inc','Cipla USA, Inc.'], 'Cipla USA Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Civica, Inc.', 'Civica Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Covis Pharma US, Inc', 'Covis Pharma')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Currax Pharmaceuticals LLC dba Cypress, Hawthorn, Macoven', 'Currax Pharmaceuticals LLC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Daiichi Sankyo, Inc.', 'Daiichi Sankyo Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(["Dr. Reddys Laboratories, Inc","Dr. Reddys Laboratories, Inc.","DR.REDDYS LABORATORIES INC","Dr.Reddys Laboratories Limited","Dr.Reddys Laboratories, Inc.","Dr. Reddy's Laboratories Inc", "Dr. Reddys Laboratories Inc.", "Dr. Reddy's Laboratories Inc.", "Dr. Reddys Laboratories Inc.,", "Dr. Reddys Laboratories Limited", "Dr. Reddy's Laboratories Limited", "Dr. Reddy's Laboratories Ltd", "Dr. Reddy's Laboratories, Inc.", "Dr.Reddys Laboratories Inc", "Dr.Reddy's Laboratories Inc", "Dr.Reddys Laboratories Inc.,", "Dr.Reddy's Laboratories Inc.,", "Dr.Reddy's Laboratories Limited", "Dr.Reddy's Laboratories Ltd", "Dr.Reddy's Laboratories Ltd.", "Dr.Reddy's Laboratories, Inc." ,'Dr. Reddys Laboratories Inc'], 'Dr Reddys Laboratories Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Endo Pharmaceuticals Solutions Inc.','Endo Pharmaceuticals, Inc.'], 'Endo Pharmaceuticals Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('ECI Pharmaceuticals, LLC', 'ECI Pharmaceuticals LLC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Edenbridge Pharmaceuticals LLC.', 'Edenbridge Pharmaceuticals, LLC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('EPI Health, LLC', 'EPI Health, Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('EYWA Pharma Inc.', 'Eywa Pharma Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Epic Pharma,LLC', 'Epic Pharma, LLC'], 'Epic Pharma LLC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Exelan Pharmaceuticals, Inc.', 'Exelan Pharmaceuticals Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('EyePoint Pharmaceuticals US, Inc', 'EyePoint Pharmaceuticals, Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Fortovia Therapeutics, Inc.', 'Fortovia Therapeutics Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Fosun Pharma USA Inc.','FOSUN PHARMA USA INC.'], 'Fosun Pharma USA Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Fresenius Kabi, USA LLC','Fresenius Medical Care North America','Fresenius Kabi USA, LLC'], 'Fresenius Medical Care Holdings, Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Genentech, Inc.', 'Genentech Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('GE Healthcare', 'GE Healthcare Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('GALDERMA LABORATORIES, L.P', 'Galderma Laboratories, L.P.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Gilead Sciences, Inc', 'Gilead Sciences')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('GlaxoSmithKline Consumer Healthcare Holdings (US) LLC', 'GlaxoSmithKline LLC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Gilead Sciences, Inc.','Gilead Sciences, LLC'], 'Gilead Sciences, Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Glenmark Pharmaceuticals, Inc','Glenmark Pharmaceuticals Limited Inc., USA','Glenmark Pharmaceutcials Inc., USA','Glenmark Pharmaceuticals Inc., USA','Glenmark Pharmaceuticals Inc. USA','Glenmark Pharmaceuticals Inc.,USA','Gilead Sciences, LLC', 'Glenmark Pharmaceutials Inc., USA','Glenamrk Pharmaceuticals Inc., USA'], 'Glenmark Pharmaceuticals Inc., USA')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('H2-Pharma, LLC', 'H2-Pharma LLC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Harris Pharmaceutical, Inc.', 'Harris Pharmaceutical, Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Heritage Pharma Labs Inc. d/b/a Avet Pharmaceuticals Labs Inc.','Heritage Pharmaceuticals Inc d/b/a as Avet Pharmaceuticals Inc','HERITAGE PHARMACEUTICALS INC D/B/A AVET PHARMACEUTICALS INC','Heritage Pharmaceuticals Inc. d/b/a Avet Pharmaceuticals Inc.'], 'Heritage Pharmaceuticals Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Hospira, Inc.','Hospira, Inc'], 'Hospira Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Hikma Pharmaceuticals USA Inc', 'Hikma Pharmaceuticals USA')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Hi-Tech Pharmacal Co., Inc.', 'Hi-Tech Pharmacal Co. Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('HRA Pharma Rare Diseases', 'HRA Pharma America, Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Heron Therapeutics, Inc.', 'Heron Therapeutics')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Horizon Therapeutics USA, Inc.', 'Horizon Therapeutics, Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Hikma Pharmaceuticals USA Inc.','Hikma Pharmceuticals USA Inc.','Hikma Specialty USA Inc.'], 'Hikma Pharmaceuticals USA Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Ingenus Pharmaceuticals, LLC', 'Ingenus Pharmaceuticals LLC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Jubilant Cadista Pharmaceuticals Inc.', 'Jubilant Cadista Pharmaceuticals Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Janssen Pharmaceuticals Inc.','Janssen Pharmaceuticals, Inc','Janssen Pharmaceuticals, Inc.','Janssen Products LP','Janssen Products, LP','Janssen Biotech, Inc.'], 'Janssen Pharmaceutical, Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['KVK-TECH, INC','KVK-TECH,INC'], 'KVK-Tech, Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('KYOWA KIRIN, INC,', 'Kyowa Kirin, Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Lannett Company, Inc.','Lannett Company Inc.'], 'Lannett Company, Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Lifestar Pharma LLC.', 'Lifestar Pharma LLC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Lupin Pharmaceuticals, Inc.','Lupin Pharmaceuticals,Inc.'], 'Lupin Pharmaceuticals, Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Mayne Pharma Inc.', 'Mayne Pharma')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['McKesson Packaging Services Business Unit of McKesson Corporation','McKesson Corporation dba SKY Packaging'], 'McKesson Corporation')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('MDD US Operations LLC, a subsidiary of Supernus Pharmaceuticals, Inc.', 'MDD US Operations, LLC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Meda Pharmaceuticals Inc.', 'Meda Pharmaceuticals')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Medimetriks Pharmaceuticals Inc','Medimetriks Pharmaceuticals, Inc.'],'Medimetriks Pharmaceuticals')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Meitheal Pharmaceuticals Inc.','Meitheal Pharmaceuticals Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Merz Pharmaceuticals, LLC','Merz North America, Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Mylan Specialty L.P.','Mylan Pharmaceuticals Inc.','Mylan Institutional LLC'], 'Mylan Institutional Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['NorthStar RxLLC','Northstar RxLLC','Northstar Rx LLC.'], 'NorthStar Rx LLC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Neos Therapeutics Brands, LLC', 'Neos Therapeutics LP')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Oceanside Pharmaceutials', 'Oceanside Pharmaceuticals')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Nexus Pharamaceuticals Inc.','Nexus Pharmaceuticals Inc.'], 'Nexus Pharmaceuticals Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Nostrum Laboratories, Inc.', 'Nostrum Laboratories Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('NIVAGEN PHARMACEUTICALS, INC.', 'Nivagen Pharmaceuticals Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Oceanside Pharmacueticals', 'Oceanside Pharmaceuticals')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Novo Nordisk Pharma, Inc.', 'Novo Nordisk')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('OrchidPharma, Inc', 'OrchidPharma Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Ohm Laboratories, Inc.', 'Ohm Laboratories Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Par Pharmaceutical, Inc.','Par Pharmaceutical, Inc','Par Pharmaceutical Inc.'], 'Par Pharmaceutical')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Pfizer Laboratories Div Pfizer Inc','Pfizer Labs, Division of Pfizer Inc.'], 'Pfizer, Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Patriot Pharmaceuticals, LLC.','Patriot Pharmaceuticals, LLC','Patriot Pharmaceuticals LLC'], 'Patriot Pharmaceuticals')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Perrigo Company','Perrigo New York Inc.'], 'Perrigo New York Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Pharmacia and Upjohn Company LLC','Pharmacia & Upjohn Company LLC'], 'Pharmacia and Upjohn Company')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('PharmaDerm, A division of Fougera Pharmaceuticals Inc.', 'PharmaDerm a division of Fougera Pharmaceuticals Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Promius Pharma, LLC.', 'Promius Pharma, LLC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Pharmaceutical Associates, Inc,', 'Pharmaceutical Associates, Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Piramal Critical Care Inc.','Piramal Critical Care, Inc.'], 'Piramal Critical Care Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Precision Dose, Inc.', 'Precision Dose Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('PruGen Pharmaceuticals', 'PruGen, Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Prasco, LLC', 'Prasco Laboratories')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('PURACAP LABORATORIES LLC DBA BLU PHARMACEUTICALS', 'PuraCap Laboratories, LLC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Rhodes Pharmaceuticals L.P.','Rhodes Pharmaceuticals L. P.'], 'Rhodes Pharmaceuticals')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Rising Pharmaceuticals, Inc.','Rising Pharma Holdings, Inc.','Rising Pharamceuticals, Inc','Rising Pharmaceuticals Inc.','Rising Pharmaceuticals, Inc'], 'Rising Health, LLC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Salix Pharmaceuticals','Salix Pharmaceuticals, Inc.','Salix Pharmaceuticals, Inc'], 'Salix Pharmaceuticals Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Sandoz Manufacturing Inc','Sandoz Inc.','SANDOZ INC.'], 'Sandoz Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['sanof-aventis U.S. LLC','Sanofi Pasteur Inc.','Sanofi-Aventis U.S. LLC'], 'Sanofi US Corporation')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Santarus, Inc..','Santarus, Inc.'], 'Santarus Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Seton Pharmaceuticals, LLC', 'Seton Pharmaceuticals')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['ScieGen Pharmaceuticals, Inc.','ScieGen Pharmaceuticals Inc','ScieGen Pharmaceuticals,Inc.'], 'ScieGen Pharmaceuticals, Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Slate Run Pharmaceuticals, LLC', 'Slate Run Pharmaceuticals')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Solco Healthcare US, LLC','Solco Healthcare US LLC','Solco Healthcare U.S., LLC','Solco healthcare U.S., LLC'], 'Solco Healthcare LLC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Strides Pharma Science Limited', 'Strides Pharma Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Sun Pharmaceutical Industries, Inc.','Sun Pharmaceutical Industries, Inc','SUN PHARMACEUTICAL INDUSTRIES, INC.','Sun Pharmaceutical Industries Inc.','Sun Pharmaceutical Industires Inc.'], 'Sun Pharmaceutical Industries Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('SUNRISE PHARMACEUTICAL, INC','Sunrise Pharmaceutical, Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Supernus Pharmaceuticals, Inc.', 'Supernus Pharmaceuticals')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('STI Pharma, LLC', 'STI Pharma LLC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Teva Neuroscience, Inc.','Teva Parenteral Medicines, Inc.','Teva Pharmaceuticals USA Inc.','Teva Pharmaceuticals USA, Inc.',"Teva Women's Health, Inc.",'Teva Respiratory, LLC','TEVA PHARMACEUTICALS USA, INC.','Teva Pharmaceuticals USA Inc','TEVA Pharmaceuticals USA Inc'], 'Teva Pharmaceuticals, Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('TruPharma, LLC','TruPharma LLC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['TAGI Pharma, Inc.','TAGI Pharma, Inc'], 'TAGI Pharma Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Taro Pharmaceuticals U.S.A., Inc.','Taro Pharmaceuticals U.S.A., inc.'], 'Taro Pharmaceuticals U.S.A., Inc')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('TerSera Therapeutics, LLC', 'TerSera Therapeutics LLC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('TIME CAP LABORATORIES, INC', 'TIME CAP LABORATORIES')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Torrent Pharma, Inc.','Torrent Pharmaceuticals Limited','TORRENT PHARMACEUTICALS LIMITED'], 'Torrent Pharma Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Upsher-Smith laboratories, LLC','Upsher-SmithLaboratories, LLC','Upsher-Smith Laboratories, LLC'], 'Upsher-Smith Laboratories, Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Virtus Pharmaceuticals LLC','Virtus Pharmaceuticals, LLC'], 'Virtus Pharmaceuticals')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Vertical Pharmaceuticals, LLC', 'Vertical Pharmaceuticals, Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Vifor (International) Inc.', 'Vifor Pharma, Inc.')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Vyera Pharmaceuticals, LLC', 'Vyera Pharmaceuticals LLC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['West-Ward Pharmaceuticals Inc.','West Ward Pharmaceuticals Corp.','West-Ward Pharmaceuticals Corp.'], 'West-Ward Pharmaceuticals Corp')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Westminster Pharmaceuticals, LLC', 'Westminster Pharmaceuticals')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Wilshire Pharmaceuticals Inc','Wilshire Pharmaceuticals, Inc.','Wilshire Pharmaceuticals Inc.'], 'Wilshire Pharmaceuticals')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Xellia Pharmaceuticals USA LLC', 'Xellia Pharmaceuticals LLC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Xspire Pharma, LLC','Xiromed, LLC'], 'Xiromed LLC')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace('Xspire Pharma, Llc', 'Xspire Pharma')
df_label['labeler_name'] = df_openFDA_NDC['labeler_name'].replace(['Zydus Pharmaceuticals USA Inc.','Zydus Pharmaceuticals (USA) Inc.'], 'Zydus Pharmaceuticals USA Inc')


In [75]:
df_openFDA_NDC = df_label

Clearing nda

In [76]:
df_openFDA_NDC['NDA #'] = df_openFDA_NDC['application_number'].str.replace(r'\D', '')
df_openFDA_NDC['NDA #'] = pd.to_numeric(df_openFDA_NDC['NDA #'])

C:\Users\sebas\AppData\Local\Temp/ipykernel_3244/746985328.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_openFDA_NDC['NDA #'] = df_openFDA_NDC['application_number'].str.replace(r'\D', '')


### Medicaid

Clearing labeler and product code

In [77]:
df_medicaid['labeler_code'] = pd.to_numeric(df_medicaid['labeler_code'])
df_medicaid['product_code'] = pd.to_numeric(df_medicaid['product_code'])

Creating a date

In [78]:
df_medicaid['date'] = 'Q' + df_medicaid['quarter'].astype(str) + ' ' + df_medicaid['year'].astype(str)
df_medicaid['date'] = pd.to_datetime(df_medicaid['date'].str.replace(r'(Q\d) (\d+)', r'\2-\1'), errors='coerce')

C:\Users\sebas\AppData\Local\Temp/ipykernel_3244/2268266489.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_medicaid['date'] = pd.to_datetime(df_medicaid['date'].str.replace(r'(Q\d) (\d+)', r'\2-\1'), errors='coerce')


### PPI

Creating a date

In [79]:
df_PPI['Period'] = df_PPI['Period'].str.replace(r'\D', '')
df_PPI['Period'] = pd.to_numeric(df_PPI['Period'])
df_PPI['day'] = 1
df_PPI['date'] = pd.to_datetime(dict(year = df_PPI['Year'], month = df_PPI['Period'], day= df_PPI['day']))

C:\Users\sebas\AppData\Local\Temp/ipykernel_3244/1936961686.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_PPI['Period'] = df_PPI['Period'].str.replace(r'\D', '')


Calculating PPI multiplier

In [80]:
df_PPI['Value'] = pd.to_numeric(df_PPI['Value'])
df_PPI['PPI_multiplier'] = df_PPI['Value'].iloc[-1] / df_PPI['Value']

# Merging raw data

### Medicaid and PPI

In [81]:
df_medicaid = pd.merge(df_medicaid, df_PPI, on = 'date', how = 'left')

Adjusting for inflation

In [82]:
df_medicaid['total_amount_reimbursed_adj'] = df_medicaid['total_amount_reimbursed'] * df_medicaid['PPI_multiplier']
df_medicaid['price_per_unit_adj'] = df_medicaid['price_per_unit'] * df_medicaid['PPI_multiplier']

Deleting redundant columns

In [83]:
df_medicaid = df_medicaid.drop(columns=['Series ID', 'Year', 'Period', 'Value', 'day', 'PPI_multiplier'])

In [84]:
len(df_medicaid)

2251895

### OpenFDA and Medicaid

Merging with Medicaid

In [85]:
df_consolidation_data = pd.merge(df_medicaid, df_openFDA_NDC, on = ['labeler_code', 'product_code'], how = 'inner')

In [86]:
len(df_consolidation_data)

549813

### OpenFDA, Medicaid and NLH

In [87]:
df_temp = df_atc

Changing the name to capital letters

In [88]:
df_temp['Source Name'] = df_temp['Source Name'].str.upper()

Removing duplicates based on source name

In [89]:
print(len(df_temp))
df_temp = df_temp.drop_duplicates(subset = ['Source Name'])
print(len(df_temp))

4016
3519


Merging on **generic name** with orig. data and removing duplicates and N/A

In [90]:
df_consolidation_data_temp = df_consolidation_data

In [91]:
df_consolidation_data_temp['generic_name'] = df_consolidation_data_temp['generic_name'].str.upper()
df_atc_generic_name_temp = pd.merge(df_consolidation_data_temp[['unique_id', 'generic_name']], df_temp[['Source Name', 'ATC (level 1)', 'ATC (level 2)', 'ATC (level 3)', 'ATC (level 4)', 'ATC (level 5)']], left_on = ['generic_name'], right_on = ['Source Name'], how = 'right')

df_atc_generic_name_temp = df_atc_generic_name_temp.dropna(subset = ['unique_id'])
df_atc_generic_name_temp = df_atc_generic_name_temp.drop_duplicates(subset = ['unique_id'])

Merging on **brand name** with orig. data and removing duplicates and N/A

In [92]:
df_consolidation_data_temp = df_consolidation_data

In [93]:
df_consolidation_data_temp['brand_name'] = df_consolidation_data_temp['brand_name'].str.upper()
df_atc_brand_name_temp = pd.merge(df_consolidation_data_temp[['unique_id', 'brand_name']], df_temp[['Source Name', 'ATC (level 1)', 'ATC (level 2)', 'ATC (level 3)', 'ATC (level 4)', 'ATC (level 5)']], left_on = ['brand_name'], right_on = ['Source Name'], how = 'right')

df_atc_brand_name_temp = df_atc_brand_name_temp.dropna(subset = ['unique_id'])
df_atc_brand_name_temp = df_atc_brand_name_temp.drop_duplicates(subset = ['unique_id'])

Concating the DataFrames for the generic and brand name

In [94]:
df_atc_generic_name_temp = df_atc_generic_name_temp.drop(columns = ['generic_name'])
df_atc_brand_name_temp = df_atc_brand_name_temp.drop(columns = ['brand_name'])

In [95]:
df_consolidation_atc_temp = pd.concat([df_atc_generic_name_temp, df_atc_brand_name_temp])

Removing duplicates based on unique id

In [96]:
print(len(df_consolidation_atc_temp))
df_consolidation_atc_temp = df_consolidation_atc_temp.drop_duplicates(subset = ['unique_id'])
print(len(df_consolidation_atc_temp))

7502
5050


Saving the consolidated data for ATC as a csv-file because it is used for assigning a running count of substitutes labelers later

In [97]:
df_consolidation_atc_temp.to_csv('raw_consolidation_data_for_ATC.csv', sep = '~', index = False)

Creating dummy variables for ATC and a list with all the names

In [98]:
df_dummy_temp = pd.get_dummies(df_consolidation_atc_temp['ATC (level 2)'])

Creating a list with ATC names for STATA

In [102]:
print('Number of ATC: ', len(list(df_dummy_temp.columns)))
print(list(df_dummy_temp.columns))

Number of ATC:  77
['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A10', 'A11', 'A12', 'A14', 'A16', 'B01', 'B02', 'B03', 'B05', 'B06', 'C01', 'C02', 'C03', 'C04', 'C05', 'C07', 'C08', 'C09', 'C10', 'D01', 'D04', 'D05', 'D06', 'D07', 'D08', 'D10', 'D11', 'G01', 'G02', 'G03', 'G04', 'H01', 'H02', 'H03', 'H04', 'H05', 'J01', 'J02', 'J04', 'J05', 'J06', 'L01', 'L02', 'L03', 'L04', 'M01', 'M03', 'M04', 'M05', 'M09', 'N01', 'N02', 'N03', 'N04', 'N05', 'N06', 'N07', 'P01', 'P02', 'P03', 'R01', 'R03', 'R05', 'R06', 'R07', 'S01', 'V03', 'V04', 'V08']


Merging with orig. data

In [61]:
df_consolidation_atc_temp = pd.merge(df_consolidation_atc_temp, df_dummy_temp, left_index = True, right_index = True, how = 'inner')

Grouping by name to include multiple ATC for each drug 

In [62]:
df_consolidation_atc_temp = df_consolidation_atc_temp.set_index(['unique_id']).groupby(level = ['unique_id']).agg('sum')
df_consolidation_atc_temp = df_consolidation_atc_temp.reset_index()

Changing the aggregated count to a dummy value

In [63]:
for (column_name, column_value) in df_consolidation_atc_temp.iteritems():
    if column_name != 'unique_id':
        df_consolidation_atc_temp[column_name] = np.where(df_consolidation_atc_temp[column_name] >= 1, 1, 0)

Merging with the consolidated data

In [64]:
df_consolidation_data = pd.merge(df_consolidation_data, df_consolidation_atc_temp, left_on = ['unique_id'], right_on = ['unique_id'], how = 'left')

In [65]:
len(df_consolidation_data)

549813

# Grouping price and quantity

Grouping price and quantity by unique id, labeler, year, and quarter and calculating price per unit adj. for the new data

In [66]:
df_temp = df_consolidation_data

In [67]:
df_temp = df_temp.set_index(['unique_id', 'labeler_name', 'year', 'quarter']).groupby(level = ['unique_id', 'labeler_name', 'year', 'quarter'])[['units_reimbursed', 'total_amount_reimbursed_adj']].agg('sum')
df_temp = df_temp.reset_index()

df_temp['price_per_unit_adj'] = df_temp['total_amount_reimbursed_adj'] / df_temp['units_reimbursed']

Dropping the price and quantity and duplicates from the old data

In [68]:
df_consolidation_data = df_consolidation_data.drop(columns = ['units_reimbursed', 'total_amount_reimbursed', 'price_per_unit', 'total_amount_reimbursed_adj', 'price_per_unit_adj'])
df_consolidation_data = df_consolidation_data.drop_duplicates(subset = ['unique_id', 'labeler_name', 'year', 'quarter'])

Merging the org. data

In [69]:
df_consolidation_data = pd.merge(df_consolidation_data, df_temp, left_on = ['unique_id', 'labeler_name', 'year', 'quarter'], right_on = ['unique_id', 'labeler_name', 'year', 'quarter'], how = 'right')

In [70]:
len(df_consolidation_data)

515574

# Downloading raw output

In [71]:
df_consolidation_data.to_csv('raw_consolidation_data.csv', sep = '~', index = False)

In [411]:
len(df_consolidation_data.drop_duplicates(subset = ['labeler_code', 'product_code'], keep = 'first'))

17625

In [412]:
len(df_consolidation_data.drop_duplicates(subset = ['unique_id'], keep = 'first'))

7214

In [415]:
df_atc.head()

,Type,RXCUI,RxNorm Name,Source ID,Source Name,ATC (level 1),ATC (level 2),ATC (level 3),ATC (level 4),ATC (level 5)
0,IN,10030.0,stannous fluoride,A01AA04,STANNOUS FLUORIDE,A,A01,A01A,A01AA,A01AA04
1,IN,10032.0,stanozolol,A14AA02,STANOZOLOL,A,A14,A14A,A14AA,A14AA02
2,IN,10109.0,streptomycin,A07AA04,STREPTOMYCIN,A,A07,A07A,A07AA,A07AA04
3,IN,10156.0,sucralfate,A02BX02,SUCRALFATE,A,A02,A02B,A02BX,A02BX02
4,IN,10174.0,sulfaguanidine,A07AB03,SULFAGUANIDINE,A,A07,A07A,A07AB,A07AB03
